In [1]:
from os import path
xml_path = path.relpath("Data/pages_8_47.xml")

from bs4 import BeautifulSoup
with open(xml_path, encoding='utf8') as file:
    contents = file.read()
    soup = BeautifulSoup(contents, 'xml')

In [2]:
import re

In [4]:
#soup.find_all('hi', string=re.compile(r'Sonntag'))

[<hi rend="letterSpaced:true;">Sonntag</hi>]

Trying to get all the text - and placeTags for Sunday:

In [3]:
start_tag = 'hi'
start_string = 'Sonntag'
end_tag = 'hi'
end_string = 'Montag.'
place_tag = 'placeName'
should_start_parsing = False

In [4]:
Sonntag = []
Places = []
for line in soup.find_all('l'):
    if line.string == start_string and line.string.parent.name == start_tag:
        should_start_parsing = True  

    if should_start_parsing:
        text = line.get_text()
        Sonntag.append(text)
        for child in line.descendants:
            if child.name == place_tag:
                for text in child.strings:
                    #hier gibts Verbesserungsbedarf! -> Ziel ist, das die strings von placeTags, die durch andere Tags unterbrochen werden, 
                    #z.B- <placeName> Schloss <persName> Esterhazy </persName> </placeName>
                    #nicht als getrennte, sondern ein string gespeichert werden. 
                    Places.append(text)          

    if line.string == end_string and line.string.parent.name == end_tag:
        should_start_parsing = False  
        break

In [7]:
#print(Sonntag)
#print(Places)

['Sonntag', 'Vormittag,', 'wo man nicht zu bestimmten Stunden in irgend eine', 'Anstalt eilen muß.', 'Wir beginnen von der k. k. Burg*). Tritt man', 'zur selben hinaus, so hat man den großen äuße=', 'ren Burgplatz vor sich, welcher durch 2 sich kreu=', 'zende Wege in 4 Rasenparterre abgetheilt ist. Rechts', 'und links führen in allen Ecken Wege auf den Wall;', 'gerade vor sich hat man das Burgthor, unter', 'Kaiser Franz I. von Nobile 1822 erbaut. Des Kaisers', '*)Alle Staatsanstalten haben das Prädikat kaiserlich=könig=', 'liche (k. k.), welches im Folgenden nur der Kürze wegen weg=', 'gelassen wird.', '5', 'Wahlspruch: Justitia Regnorum Fundamentum, ziert', 'die Stadtseite. Es ist 38 Klafter lang, und hat fünf', 'Durchgänge, welche durch 18 dorische Säulen gebil=', "det werden, die 4' 7'' Durchmesser haben. Die mitt=", 'lere Durchfahrt ist dem allerhöchsten Hofe vorbehal=', 'ten. Zu beiden Seiten des Gebäudes führen auch', 'Stiegen auf den Wall, die aber gewöhnlich verschlos=', 'sen s

Cleaning and structuring the list of places:

In [5]:
while(" " in Places):
    Places.remove(" ")
print(Places) 

['k. k. Burg', 'Burgplatz', 'Burgthor', 'kaiserlichen Stallungen', 'Ge=', 'bäude der ungarischen Garde', 'Palais des ', 'Fürsten Auersperg', 'Gebäude des ', 'Geographi=', 'schen (Militär=)Jnstituts', 'Kriminal=', 'Gebäude', 'Kahlengebirge', 'Mariahilfer Hauptstraße', 'Schön=', 'brunn', 'Grena=', 'dierkaserne', 'Getreide=', 'markt', 'artesischen Brunnen', 'Akazien=Alleen', 'Wien', 'Kettenstege', 'der steinernen Brücke', 'Polytechnische Jnstitut', 'Karlskirche', 'Burg', 'Ritter=', 'Ceremoniensaal', 'Appartements ', 'Sr Majestät des Kaisers', 'Rittersaal', 'Antiken=', 'Mineralienkabinet', 'astronomische Thurm', 'Burg', 'Hofbiblio=', 'thek=Gebäudes', 'Josephsplatze', 'Palais des ', 'Erzherzogs Karl', 'Burg=', 'platze', 'Volksgartens', 'Burg=', 'gartens', 'Burggarten', 'Gewächshäuser', 'Wall', 'Palais des ', 'Erzherzogs Karl', 'Spitalplatze', 'Hofbau=', 'amte', 'Kärnthnerthor=Theater', 'Wall', 'Kärnthnerthore', 'Kärnthnerstraße', 'Wieden', 'Baden', 'Starhembergische ', 'Freihaus', 'Polytech

In [23]:
clean_places = []
for e in Places:
    if clean_places and clean_places[-1].endswith('='):
        clean_places[-1] = clean_places[-1][:-1] + e
    elif clean_places and (clean_places[-1].endswith('des ') or clean_places[-1].endswith('des')):
        clean_places[-1] = clean_places[-1] + ' ' + e
    elif clean_places and (clean_places[-1].endswith('der ') or clean_places[-1].endswith('der')): 
        clean_places[-1] = clean_places[-1] + ' ' + e     
    elif clean_places and clean_places[-1].endswith('den '): 
        clean_places[-1] = clean_places[-1] + ' ' + e 
    elif clean_places and (clean_places[-1].endswith('Fürsten ') or clean_places[-1].endswith('.')):
        clean_places[-1] = clean_places[-1] + ' ' + e         
    else:
        clean_places.append(e)


In [24]:
print(clean_places)

['k. k. Burg', 'Burgplatz', 'Burgthor', 'kaiserlichen Stallungen', 'Gebäude der ungarischen Garde', 'Palais des  Fürsten Auersperg', 'Gebäude des  Geographischen (Militär=)Jnstituts', 'KriminalGebäude', 'Kahlengebirge', 'Mariahilfer Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Getreidemarkt', 'artesischen Brunnen', 'Akazien=Alleen', 'Wien', 'Kettenstege', 'der steinernen Brücke', 'Polytechnische Jnstitut', 'Karlskirche', 'Burg', 'RitterCeremoniensaal', 'Appartements ', 'Sr Majestät des Kaisers', 'Rittersaal', 'AntikenMineralienkabinet', 'astronomische Thurm', 'Burg', 'Hofbibliothek=Gebäudes Josephsplatze', 'Palais des  Erzherzogs Karl', 'Burgplatze', 'Volksgartens', 'Burggartens', 'Burggarten', 'Gewächshäuser', 'Wall', 'Palais des  Erzherzogs Karl', 'Spitalplatze', 'Hofbauamte', 'Kärnthnerthor=Theater', 'Wall', 'Kärnthnerthore', 'Kärnthnerstraße', 'Wieden', 'Baden', 'Starhembergische ', 'Freihaus', 'Polytechnische Jnstitut', 'Karlskirche', 'Starhembergische Frelhaus', 'Wienerwaldes

Some manual refinements, necessary (hopefully) only in this case: 

In [25]:
import functools

In [26]:
clean_places.remove('ist Wien = Donau?')
clean_places.remove('Vorstadt = Bezirk')
clean_places.index('Wienerwaldes Schneeberg')

50

put together: große Steinbrücke, Starhembergisches Freihaus, Appartements Sr. Majestät, 'fürstlich''Esterhazische sogenannte ''rothe Haus'
seperated: Hofbibliotheksgebäude - Josephplatz; Wienerwaldes - Schneeberg; 

In [27]:
clean_places[73:75] = [functools.reduce(lambda x, y: x + ' ' + y, clean_places[73:75])]
print(clean_places[73])

große Steinbrücke


In [28]:
clean_places[45:47] = [functools.reduce(lambda x, y: x + y, clean_places[45:47])]
print(clean_places[45])

Starhembergische Freihaus


In [29]:
clean_places[22:24] = [functools.reduce(lambda x, y: x + y, clean_places[22:24])]
print(clean_places[22])

Appartements Sr Majestät des Kaisers


In [30]:
clean_places[154:157] = [functools.reduce(lambda x, y: x + y, clean_places[154:157])]
print(clean_places[154])

fürstlichEsterhazische sogenannte rothe Haus


In [31]:
clean_places.insert(27, 'Hofbibliotek=Gebäudes')
clean_places.insert(28, 'Josephsplatze')
clean_places.remove('Hofbibliothek=Gebäudes Josephsplatze')

In [32]:
clean_places.insert(49, 'Wienerwaldes')
clean_places.insert(50, 'Schneeberg')
clean_places.remove('Wienerwaldes Schneeberg')

In [33]:
clean_places[85:87] = [functools.reduce(lambda x, y: x + ' ' + y, clean_places[85:87])]
print(clean_places[85])

Kloster und Kirche der  Dominikaner


In [39]:
clean_places_text = open('Data/clean_places_text.txt','w', encoding='utf8')
for item in clean_places:
	clean_places_text.write(item+"\n")
clean_places_text.close()

There are several possibilities to normalize text - one idea could be, using the spacy-package, to lemmatize the places - the problem here although is that spacy splits words that belong together - as names - and the results of lemmatization themselves do not look too good ("Stallunge")

In [1]:
#import spacy
#nlp = spacy.load("de_core_news_sm")

In [16]:
#places_text = ' '.join(clean_places)
#nlp_places = nlp(places_text)

In [17]:
#lemma_list = []
#for word in nlp_places: 
    #lemma = word.lemma_
    #lemma_list.append(lemma)    
#print(lemma_list)

['k.', 'k.', 'Burg', 'Burgplatz', 'Burgthor', 'kaiserlichen', 'Stallunge', 'Gebäude', 'der', 'ungarisch', 'Garde', 'Palais', 'der', 'Fürst', 'Auersperg', 'Gebäude', 'der', 'Geographische', '--', 'Militär=)Jnstituts', 'KriminalGebäude', 'Kahlengebirge', 'Mariahilfer', 'Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Getreidemarkt', 'artesisch', 'Brunnen', 'Akazie', '=', 'Alleen', 'Wien', 'Kettenstege', 'der', 'steinern', 'Brücke', 'Polytechnische', 'Jnstitut', 'karlskirch', 'Burg', 'RitterCeremoniensaal', 'Appartement', ' ', 'Sr', 'Majestät', 'der', 'Kaiser', 'Rittersaal', 'AntikenMineralienkabinet', 'astronomisch', 'Thurm', 'Burg', 'Hofbibliothek', '=', 'GebäudesJosephsplatze', 'Palais', 'der', 'Erzherzog', 'Karl', 'Burgplatze', 'Volksgarten', 'Burggarten', 'Burggarten', 'Gewächshäuser', 'Wall', 'Palais', 'der', 'Erzherzog', 'Karl', 'Spitalplatze', 'Hofbauamen', 'Kärnthnerthor', '=', 'Theater', 'Wall', 'Kärnthnerthore', 'Kärnthnerstraße', 'Wiede', 'Baden', 'Starhembergische', ' ', 'Fr

Another Option would be fuzzy string matching 

In [35]:
import difflib as dl

In [36]:
Similiar_words = []
for word in clean_places:
    close_words = dl.get_close_matches(word, clean_places, cutoff= 0.9)
    Similiar_words.append(close_words)

In [37]:
place_list = []
for sublist in Similiar_words:
    sublist = sorted(sublist)
    place = sublist[0]
    place_list.append(place)

In [40]:
print(len(place_list))
print(place_list)
place_list_text= open('Data/place_list_text.txt','w', encoding='utf8')
for item in place_list:
	place_list_text.write(item+"\n")
place_list_text.close()


188
['k. k. Burg', 'Burgplatz', 'Burgthor', 'kaiserlichen Stallungen', 'Gebäude der ungarischen Garde', 'Palais des  Fürsten Auersperg', 'Gebäude des  Geographischen (Militär=)Jnstituts', 'KriminalGebäude', 'Kahlengebirge', 'Mariahilfer Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Getreidemarkt', 'artesischen Brunnen', 'Akazien=Alleen', 'Wien', 'Kettenstege', 'der steinernen Brücke', 'Polytechnische Jnstitut', 'Karlskirche', 'Burg', 'RitterCeremoniensaal', 'Appartements Sr Majestät des Kaisers', 'Rittersaal', 'AntikenMineralienkabinet', 'astronomische Thurm', 'Burg', 'Hofbibliotek=Gebäudes', 'Josephsplatze', 'Palais des  Erzherzogs Karl', 'Burgplatz', 'Volksgarten', 'Burggarten', 'Burggarten', 'Gewächshäuser', 'Wall', 'Palais des  Erzherzogs Karl', 'Spitalplatze', 'Hofbauamte', 'Kärnthnerthor=Theater', 'Wall', 'Kärnthnerthore', 'Kärnthnerstraße', 'Wieden', 'Baden', 'Starhembergische Freihaus', 'Polytechnische Jnstitut', 'Karlskirche', 'Starhembergische Freihaus', 'Wienerwaldes', 'S

It worked pretty well - but, we still have some cases like "Sommerpalais des Fürsten Schwarzenberg" and "Schwarzenbergisches Palais", which are the same, but have not been matched...

Next is to create a dataframe containing the coordinates for each place - i'm trying to use the df from the Wien Geschichte Wiki (https://www.data.gv.at/katalog/dataset/wien-geschichte-wiki#resources) containing gis-data for historic places and sights in vienna

In [121]:
import pandas as pd
csv_path = path.relpath("Data/KULTURWIKIOGD.csv")
usecolumns = ['OBJECTID', 'SHAPE', 'ADRESSE', 'SEITENNAME']
coordinates = pd.read_csv(csv_path,  index_col="OBJECTID", usecols=usecolumns)

In [43]:
coordinates.head()

,SHAPE,ADRESSE,SEITENNAME
OBJECTID,,,
294006,POINT (16.24449608644173 48.20765995647323),"14., Mondweg 91",Franz-Sauer-Park
294007,POINT (16.258093352548578 48.19709893851195),"13., Rußpekgasse 3",Franz-Schimon-Park
294008,POINT (16.26067781801285 48.182308544859616),"13., Schweizertalstraße 29D",Franz-Schmidt-Park
294009,POINT (16.338812997907947 48.188874975260035),"15., U-Bahn-Bogen 1",Franz-Schwarz-Park
294010,POINT (16.39077256132719 48.22313059046534),"02., Nordbahnstraße 49",Franziska-Löw-Park


In [44]:
check = coordinates[coordinates['SEITENNAME'].str.contains('Stephan')]

In [45]:
print(check)

                                                  SHAPE  \
OBJECTID                                                  
286521     POINT (16.372830852928722 48.20800654886508)   
286243      POINT (16.37495718182766 48.21430702051684)   
286244     POINT (16.373247181361492 48.20841804972867)   
286245    POINT (16.355080020424758 48.190544371399504)   
286679     POINT (16.375844996927345 48.20779753290685)   
283015     POINT (16.372781185569394 48.20901404694178)   
284277     POINT (16.337031664414706 48.27482055389401)   
284278     POINT (16.372107083378758 48.20811037952966)   
284279     POINT (16.372960181555417 48.20768305339394)   
285422     POINT (16.365136215578936 48.20448507297512)   
285423      POINT (16.30201549575732 48.18407421016061)   
284588     POINT (16.358538612065974 48.20127775630736)   
322024      POINT (16.37540812300062 48.21532973453562)   
322025      POINT (16.37540812300062 48.21532973453562)   
292820    POINT (16.280448536780664 48.206599327280664) 

In [46]:
from thefuzz import process, fuzz 

In [122]:
coordinates['MATCH_SCORE'] = coordinates['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1])

In [114]:
coordinates = coordinates.sort_values('MATCH_SCORE', ascending=False)

In [131]:
coordinates_choice = coordinates.drop(coordinates[coordinates['MATCH_SCORE'] <= 79].index)

In [132]:
coordinates_choice['NEUE_NAMEN'] = coordinates_choice['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])

In [133]:
coordinates_choice

,SHAPE,ADRESSE,SEITENNAME,MATCH_SCORE,NEUE_NAMEN
OBJECTID,,,,,
293862,POINT (16.375805904211965 48.23846267177948),"20., Leithastraße 16",Brigittenau,100,Brigittenau
293873,POINT (16.36595721226683 48.20455507212498),"01., Burggarten P Burggartenrestaurant",Burggarten,100,Burggarten
293903,POINT (16.376012013073886 48.213567135220664),"02., Obere Donaustraße 93",Dianabadufer,80,Dianabad
293908,POINT (16.416258411226355 48.22030842223218),"02., Kafkasteg R RESTAURANT",Donau,100,Donau
293914,POINT (16.3950020299091 48.21135123303799),"02., Schüttelstraße 17A",Donaukanal,100,Donaukanal
...,...,...,...,...,...
294357,POINT (16.362861238868568 48.2125940358467),"01., Mölker Bastei 14",Mölker Steig,83,Mölkerbastei
294365,POINT (16.362362219955745 48.197948106357586),"06., Naschmarkt STAND 370-373",Naschmarkt,80,Fischmarkt
294500,POINT (16.374430808795385 48.21228542778926),"01., Morzinplatz 1",Ruprechtsstiege,85,St. Ruprechtstiege


Das Problem ist ja, dass ein Ort wie Ferdinands-Brücke im Prinzip dasselbe ist wie Ferdinandsbrücke, aber im Vergleich einen niederigen Score hat...

In [116]:
mask = coordinates_choice['SEITENNAME'].isin(place_list)

In [117]:
place_coord = coordinates_choice[mask]

Am Ende matcht der Code aber dann doch nur die 100-prozentigen Matches ... 

35


In [51]:
place_coord.to_csv('Data/Sonntag_coord.csv', sep=',', index=False, encoding='utf-8')

Nächster Versuch mit dem Gazeteer Steinhausen Plan: 

In [55]:
xlsx_path = path.relpath("Data/Gazetteer_Steinhausenplan_V5.xlsx")
usecolumns = ['ID_neu', 'Toponym', 'Sicherheit', 'Longitude', 'Latitude']
Steinhausen_coordinates = pd.read_excel(xlsx_path, usecols=usecolumns, index_col=0)  

In [56]:
Steinhausen_coordinates.head()

,Toponym,Sicherheit,Longitude,Latitude
ID_neu,,,,
1,3 Cron:,hoch,16.376307,48.211565
2,3 Fisch,hoch,16.374240,48.210826
3,3 Fisch.,hoch,16.374100,48.210932
4,3 Rueben,mittel,16.370820,48.208375
5,5. Cronen,hoch,16.368902,48.211668


In [57]:
Steinhausen_coordinates = Steinhausen_coordinates[Steinhausen_coordinates['Sicherheit'].str.contains('hoch')]

In [58]:
Steinhausen_coordinates.head()

,Toponym,Sicherheit,Longitude,Latitude
ID_neu,,,,
1,3 Cron:,hoch,16.376307,48.211565
2,3 Fisch,hoch,16.374240,48.210826
3,3 Fisch.,hoch,16.374100,48.210932
5,5. Cronen,hoch,16.368902,48.211668
6,7. Stern,hoch,16.373567,48.211813


In [66]:
mask2 = Steinhausen_coordinates['Toponym'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])
print(mask2)

ID_neu
1       True
2       True
3       True
5       True
6       True
        ... 
1402    True
1406    True
1407    True
1408    True
1442    True
Name: Toponym, Length: 995, dtype: bool


In [64]:
place_coord_SH = Steinhausen_coordinates[mask2]

In [65]:
print(place_coord_SH)

Empty DataFrame
Columns: [Toponym, Sicherheit, Longitude, Latitude]
Index: []
